In [95]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import os

from keras.layers import Input,Embedding,Flatten,Dot,Dense
from keras.models import Model

%matplotlib inline

In [96]:
dataset = pd.read_csv(r'C:\Users\Kristaps\Desktop\dataset\ml-latest-small\ratings.csv')
dataset.head(6)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400


In [90]:
movies = pd.read_csv(r'C:\Users\Kristaps\Desktop\dataset\ml-latest-small\movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [79]:
dataset.shape


(100836, 4)

In [91]:
movies.movieId = movies.movieId.astype('category').cat.codes.values
movies.head()

,movieId,title,genres
0,0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,Jumanji (1995),Adventure|Children|Fantasy
2,2,Grumpier Old Men (1995),Comedy|Romance
3,3,Waiting to Exhale (1995),Comedy|Drama|Romance
4,4,Father of the Bride Part II (1995),Comedy


In [92]:
reindexedmovies = movies.to_csv(r'C:\Users\Kristaps\Desktop\dataset\ml-latest-small\reindexedmovies.csv', index = True)


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Kristaps\\Desktop\\dataset\\ml-latest-small\\reindexedmovies.csv'

In [80]:
  dataset.userId = dataset.userId.astype('category').cat.codes.values
  dataset.movieId = dataset.movieId.astype('category').cat.codes.values
  #reindexed = dataset.to_csv(r'C:\Users\Kristaps\Desktop\dataset\ml-latest-small\reindexed.csv', index = True)
  dataset.head(50)

,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,2,4.0,964981247
2,0,5,4.0,964982224
3,0,43,5.0,964983815
4,0,46,5.0,964982931
5,0,62,3.0,964982400
6,0,89,5.0,964980868
7,0,97,4.0,964982176
8,0,124,5.0,964984041
9,0,130,5.0,964984100


In [34]:
# dataset['rating'] = dataset['rating'].map({1:-1,2:-1,3:0,4:1,5:1,1.5:-1,2.5:-1,3.5:1,4.5:1})

In [81]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(dataset,test_size=0.2, random_state =5)

In [82]:
train.head()


,userId,movieId,rating,timestamp
96041,601,312,3.0,840876228
21842,140,7061,4.0,1513130002
66322,426,3718,2.5,1053070768
60779,390,3801,4.0,1030824826
6435,43,58,3.0,869252497


In [97]:
n_users, n_movies = len(dataset.userId.unique()), len(dataset.movieId.unique())
print(n_users,n_movies)

610 9724


In [59]:
userembedding = dataset['userId'].max()
movieembedding = dataset['movieId'].max()
print(userembedding,movieembedding)

610 193609


In [61]:
n_movies1 = dataset['movieId'].nunique()
n_users1 = dataset['userId'].nunique()
print(n_users1,n_movies1)

610 9724


In [101]:
user_input = Input(shape=[1], name='Users')
user_embedding = Embedding(n_users+1, 8, name="Embedded_Users")(user_input)
user_final = Flatten(name='Flaten_User')(user_embedding)

movie_input = Input(shape=[1], name='Movies')
movie_embedding = Embedding(n_movies+1, 8, name="Embedded_Movies")(movie_input)
movie_final = Flatten(name='Flaten_Movie')(movie_embedding)

mult = Dot(name='Dot_product', axes=1)([user_final,movie_final])
model = Model([user_input,movie_input],mult)
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Users (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
Movies (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
Embedded_Users (Embedding)      (None, 1, 8)         4888        Users[0][0]                      
__________________________________________________________________________________________________
Embedded_Movies (Embedding)     (None, 1, 8)         77800       Movies[0][0]                     
____________________________________________________________________________________________

In [103]:
from keras.models import load_model
if os.path.exists(r'C:\Users\Kristaps\Desktop\recsys\movie_rec.m5'):
    model = load_model(r'C:\Users\Kristaps\Desktop\recsys\movie_rec.m5')
else:
    history = model.fit([train.userId, train.movieId], train.rating, epochs=100,verbose=1)
    model.save(r'C:\Users\Kristaps\Desktop\recsys\movie_rec.m5')
    plt.plot(history.history['loss'])
    plt.xlabel = ('Epochs')
    plt.ylabel = ('Training_Error')

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
model.evaluate([test.userId,test.movieId], test.rating)

In [ ]:
predictions = model.predict([test.userId.head(20),test.movieId.head(20)])
[print(predictions[i], test.rating.iloc[i]) for i in range(0,20)]

In [ ]:
moviedata = np.array(list(set(dataset.movieId)))
moviedata[:8]

In [ ]:
user = np.array([1 for i in range(len(moviedata))])
user[:8]

In [ ]:
predictions = model.predict([user,moviedata])
predictions = np.array([a[0] for a in predictions])

recommendedmovies = (-predictions).argsort()[:8]
recommendedmovies

In [221]:
pd.DataFrame(train).max()

userId       6.100000e+02
movieId      1.936090e+05
rating       5.000000e+00
timestamp    1.537799e+09
dtype: float64